In [1]:
import pandas as pd
import ast

In [2]:
df = pd.read_csv("full_TRAINING_DATASET.csv")

In [3]:
df

,label,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,Ever get the feeling your life circles the rou...
2,0,"Why the Truth Might Get You Fired October 29, ..."
3,0,Print \nAn Iranian woman has been sentenced to...
4,1,"In these trying times, Jackie Mason is the Voi..."
...,...,...
69293,1,There are a larger number of shark attacks in ...
69294,1,Democrats have now become the party of the [At...
69295,1,Says an alternative to Social Security that op...
69296,0,On lifting the U.S. Cuban embargo and allowing...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from time import time

# Load tokenizer and model
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

c:\Users\matej\SKOLA\Diplomovka\Code\EmbeddingModels\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
import re


def preprocess_text(text):
    text = re.sub(r"[‘’]", "'", text)
    return re.sub(r"[“”`]", '"', text)


def tokenize_text(text):
    return tokenizer(text, return_tensors="pt", truncation=False)


def chunk_and_pad_tokenized_input(inputs, max_length=512):

    input_ids = inputs["input_ids"][0]
    attention_mask = inputs["attention_mask"][0]
    chunks = []

    for i in range(0, len(input_ids), max_length):
        chunk_input_ids = input_ids[i : i + max_length]
        chunk_attention_mask = attention_mask[i : i + max_length]

        if len(chunk_input_ids) < max_length:
            padding_length = max_length - len(chunk_input_ids)
            chunk_input_ids = torch.cat(
                [
                    chunk_input_ids,
                    torch.tensor([tokenizer.pad_token_id] * padding_length),
                ]
            )
            chunk_attention_mask = torch.cat(
                [chunk_attention_mask, torch.tensor([0] * padding_length)]
            )

        chunks.append(
            {
                "input_ids": chunk_input_ids.unsqueeze(0),
                "attention_mask": chunk_attention_mask.unsqueeze(0),
            }
        )

    return chunks


def get_embeddings_and_average(chunks, padding_token_id=0):
    all_embeddings = []
    for chunk in chunks:
        chunk = {key: value.to("cuda") for key, value in chunk.items()}

        with torch.no_grad():
            outputs = model(**chunk)

        token_embeddings = outputs.last_hidden_state

        attention_mask = chunk["attention_mask"]

        padding_positions = (attention_mask == padding_token_id).nonzero(as_tuple=True)[
            1
        ]
        if len(padding_positions) > 0:
            first_padding_position = padding_positions[0].item()

            token_embeddings = token_embeddings[:, :first_padding_position, :]
        all_embeddings.append(token_embeddings)

    all_embeddings = torch.cat(all_embeddings, dim=1)

    final_embedding = all_embeddings.mean(dim=1)

    return final_embedding


def get_roberta_embedding(text, max_length=512):

    text = preprocess_text(text)
    inputs = tokenize_text(text)

    chunks = chunk_and_pad_tokenized_input(inputs, max_length)

    final_embedding = get_embeddings_and_average(chunks)

    return final_embedding[0].cpu().numpy()

In [5]:
text = df.iloc[1]["text"]
text = preprocess_text(text)
text

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed "no debate so far as I could ascertain as to who their spokesman was to be" (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). "I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us," Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [ ]:
example = tokenizer(
    "Random tokenizer text example.", return_tensors="pt", truncation=False
)
example

{'input_ids': tensor([[    0, 45134, 19233,  6315,  2788,  1246,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [23]:
tokenizer.convert_ids_to_tokens(example["input_ids"][0])

['<s>', 'Random', 'Ġtoken', 'izer', 'Ġtext', 'Ġexample', '.', '</s>']

In [6]:
out = tokenize_text(text)
out

Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[    0, 27172,   120,     5,  2157,   110,   301, 13820,     5,  1062,
          9006,  1195,    87,  3885,    11,    10,  1359,   516,  1706,     5,
          3833,  6381,   116,   646, 38655,  2235,  1189,     5,   380,   693,
            15,  2894,    11, 14610,   219,     6,  6176,  2647, 36664,     6,
          5517,     4, 37691,  1493,  2834,    69,   144,   533,     7,   218,
            69,  9714,  3588,    13,     5, 11059,     9,    69,   360,     5,
           169,  4523, 22989,  1173,   424,  6000,  5328,    14,  3312,  3588,
             4,  1437,  3580,     9,  2860, 12809,  1635,     6,  5141, 46581,
         41031,  9725,    19,   106,  2929,   107,   536,    77,    79,    78,
          4873,    10,  2647, 36664, 15128,  1380,     4,    20,   394,     9,
             5,  1564,  3978,   167,  4366,    11, 15077,    14,     5,   521,
           956,    22,  2362,  2625,    98,   444,    25,    38,   115, 29477,
            25,     7,    54,    49,  

In [9]:
chunks = chunk_and_pad_tokenized_input(out)
chunks

[{'input_ids': tensor([[    0, 27172,   120,     5,  2157,   110,   301, 13820,     5,  1062,
            9006,  1195,    87,  3885,    11,    10,  1359,   516,  1706,     5,
            3833,  6381,   116,   646, 38655,  2235,  1189,     5,   380,   693,
              15,  2894,    11, 14610,   219,     6,  6176,  2647, 36664,     6,
            5517,     4, 37691,  1493,  2834,    69,   144,   533,     7,   218,
              69,  9714,  3588,    13,     5, 11059,     9,    69,   360,     5,
             169,  4523, 22989,  1173,   424,  6000,  5328,    14,  3312,  3588,
               4,  1437,  3580,     9,  2860, 12809,  1635,     6,  5141, 46581,
           41031,  9725,    19,   106,  2929,   107,   536,    77,    79,    78,
            4873,    10,  2647, 36664, 15128,  1380,     4,    20,   394,     9,
               5,  1564,  3978,   167,  4366,    11, 15077,    14,     5,   521,
             956,    22,  2362,  2625,    98,   444,    25,    38,   115, 29477,
              2

In [10]:
len(chunks[-1]["input_ids"][0])

512

In [ ]:
chunks[-1]["input_ids"]

In [11]:
embedding = get_embeddings_and_average(chunks)

In [46]:
embedding.shape

torch.Size([1, 768])

In [31]:
len(embedding[0])

768

In [34]:
len(embedding.cpu().numpy())

1

In [47]:
len(out["input_ids"][0])

887

In [58]:
df["article_embedding"] = df["text"].apply(lambda x: get_roberta_embedding(x))

In [62]:
df

,label,text,article_embedding
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,"[0.0028758862, 0.025872586, 0.03593416, -0.099..."
1,1,Ever get the feeling your life circles the rou...,"[0.032413572, 0.035666395, 0.057574242, -0.100..."
2,0,"Why the Truth Might Get You Fired October 29, ...","[0.037595626, 0.05027227, 0.060264364, -0.0506..."
3,0,Print \nAn Iranian woman has been sentenced to...,"[0.028826052, 0.057950247, -0.00472429, -0.082..."
4,1,"In these trying times, Jackie Mason is the Voi...","[-0.02413057, 0.03461668, 0.060766198, -0.1634..."
...,...,...,...
69293,1,There are a larger number of shark attacks in ...,"[0.038691726, 0.14385675, 0.050323993, -0.0680..."
69294,1,Democrats have now become the party of the [At...,"[-0.07634911, 0.035728134, 0.058021225, -0.027..."
69295,1,Says an alternative to Social Security that op...,"[0.04432549, 0.11710828, 0.010645155, -0.05531..."
69296,0,On lifting the U.S. Cuban embargo and allowing...,"[0.0734633, 0.14546232, 0.10696487, -0.0214499..."


In [63]:
df = df.dropna()

In [64]:
df

,label,text,article_embedding
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,"[0.0028758862, 0.025872586, 0.03593416, -0.099..."
1,1,Ever get the feeling your life circles the rou...,"[0.032413572, 0.035666395, 0.057574242, -0.100..."
2,0,"Why the Truth Might Get You Fired October 29, ...","[0.037595626, 0.05027227, 0.060264364, -0.0506..."
3,0,Print \nAn Iranian woman has been sentenced to...,"[0.028826052, 0.057950247, -0.00472429, -0.082..."
4,1,"In these trying times, Jackie Mason is the Voi...","[-0.02413057, 0.03461668, 0.060766198, -0.1634..."
...,...,...,...
69293,1,There are a larger number of shark attacks in ...,"[0.038691726, 0.14385675, 0.050323993, -0.0680..."
69294,1,Democrats have now become the party of the [At...,"[-0.07634911, 0.035728134, 0.058021225, -0.027..."
69295,1,Says an alternative to Social Security that op...,"[0.04432549, 0.11710828, 0.010645155, -0.05531..."
69296,0,On lifting the U.S. Cuban embargo and allowing...,"[0.0734633, 0.14546232, 0.10696487, -0.0214499..."


In [65]:
import numpy as np


df["article_embedding"] = df["article_embedding"].apply(
    lambda x: x.tolist() if isinstance(x, np.ndarray) else x
)

In [66]:
df.to_csv(
    "embedded_datasets/RoBERTa/RoBERTa_Embedded_Dataset.csv",
    encoding="utf-8",
    index=False,
)

# Generalization dataset


In [17]:
import pandas as pd
import ast


df = pd.read_csv("full_GENERALIZATION_DATASET.csv")

In [18]:
df

,label,text
0,0,"Daniel Greenfield, a Shillman Journalism Fello..."
1,0,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,1,U.S. Secretary of State John F. Kerry said Mon...
3,0,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,1,It's primary day in New York and front-runners...
...,...,...
5985,1,The State Department told the Republican Natio...
5986,0,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
5987,0,Anti-Trump Protesters Are Tools of the Oligar...
5988,1,"ADDIS ABABA, Ethiopia —President Obama convene..."


In [7]:
df["article_embedding"] = df["text"].apply(lambda x: get_roberta_embedding(x))

Token indices sequence length is longer than the specified maximum sequence length for this model (1549 > 512). Running this sequence through the model will result in indexing errors


In [8]:
df = df.dropna()

In [11]:
df

,label,text,article_embedding
0,0,"Daniel Greenfield, a Shillman Journalism Fello...","[-0.002649596, 0.061043363, 0.052470032, -0.09..."
1,0,Google Pinterest Digg Linkedin Reddit Stumbleu...,"[0.0022733945, 0.03975339, 0.039920527, -0.113..."
2,1,U.S. Secretary of State John F. Kerry said Mon...,"[0.054484554, 0.11057529, 0.06774962, -0.06846..."
3,0,"— Kaydee King (@KaydeeKing) November 9, 2016 T...","[-0.027517661, 0.05548241, 0.016553458, -0.064..."
4,1,It's primary day in New York and front-runners...,"[0.004216633, 0.09147257, 0.037530564, -0.0814..."
...,...,...,...
5985,1,The State Department told the Republican Natio...,"[0.0014618421, 0.11570988, 0.020603774, -0.064..."
5986,0,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,"[0.014944622, 0.077357404, 0.10143796, -0.0913..."
5987,0,Anti-Trump Protesters Are Tools of the Oligar...,"[-0.007107026, 0.07709258, 0.08062994, -0.1241..."
5988,1,"ADDIS ABABA, Ethiopia —President Obama convene...","[0.03555587, 0.088559456, 0.043276478, -0.0736..."


In [12]:
import numpy as np


df["article_embedding"] = df["article_embedding"].apply(
    lambda x: x.tolist() if isinstance(x, np.ndarray) else x
)

In [13]:
df.to_csv(
    "embedded_datasets/RoBERTa/RoBERTa_Embedded_Generalization_Dataset.csv",
    encoding="utf-8",
    index=False,
)